In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q --upgrade huggingface_hub
!pip -q install datasets
!pip -q install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
# Configure BitsAndBytesConfig for loading data in 4-bit format with double quantization using nf4 method
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
from huggingface_hub import notebook_login

notebook_login()

#LLama-3 Model

In [5]:
device = "cuda"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [7]:
messages = [
    {"role": "system", "content": "you're a skilled financial analyst specialized in detecting market sentiment from news sources."},
    {"role": "user", "content": "Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.\
      Return only a single word, either Positive or Negative or Neutral:\n sentence: Copper, lead and nickel also dropped... HBOS ( HBOS ) plummeted 20 % to 70.3 pence after saying this year??, label: "},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Negative


Below is the prompt format for Llama 3 chat model


![picture](https://drive.google.com/uc?export=view&id=1r15q533qUw2S884NDJafOCDH3HcQJtxM)


In [22]:
import re

def extract_sentiment_word(sentence):
  words = sentence.split()
  last_word = words[-1].strip('[]"')
  return last_word

prompt_template = """
 <|start_header_id|>system<|end_header_id|>
  you're a skilled financial analyst specialized in detecting market sentiment from news sources.<|eot_id|><|start_header_id|>user<|end_header_id|>

  Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.\n
  Return only a single word, either Positive or Negative or Neutral:

  sentence: {query}
  label: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

  """
query="Copper, lead and nickel also dropped... HBOS ( HBOS ) plummeted 20 % to 70.3 pence after saying this year?"

prompt = prompt_template.format(query=query)

encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id= tokenizer.eos_token_id, eos_token_id=terminators)

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


<|begin_of_text|>
 <|start_header_id|>system<|end_header_id|>
  you're a skilled financial analyst specialized in detecting market sentiment from news sources.<|eot_id|><|start_header_id|>user<|end_header_id|>

  Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.

  Return only a single word, either Positive or Negative or Neutral:

  sentence: Copper, lead and nickel also dropped... HBOS ( HBOS ) plummeted 20 % to 70.3 pence after saying this year?
  label: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

   Negative<|eot_id|>


In order to retrieve just the response you need to slice the output at the index of the input tokens. Like so using input_token_len:

In [21]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_token_len = input_ids.shape[-1]
outputs = model.generate(input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0][input_token_len:]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1542: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 Negative<|eot_id|>


#Phi-3

#**TFSN**

In [ ]:
import pandas as pd

In [ ]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment" )
dataset=dataset.shuffle(seed=42)

In [ ]:
import re
def remove_url(text):
    return re.sub(r'https://\S+', '', text)

In [ ]:
dataset['train'] = dataset['train'].map(lambda example: {'text': remove_url(example['text'])})
dataset['validation'] = dataset['validation'].map(lambda example: {'text': remove_url(example['text'])})

In [ ]:
dataset['train'][0]

In [ ]:
# Mapping labels
label_mapping = {0: 'Negative', 2: 'Neutral', 1: 'Positive'}
dataset['train'] = dataset['train'].map(lambda example: {'label': label_mapping[example['label']], 'text': example['text']})

In [ ]:
dataset['validation'] = dataset['validation'].map(lambda example: {'label': label_mapping[example['label']], 'text': example['text']})

In [ ]:
dataset['validation'][0]

In [ ]:
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['validation'])

In [ ]:
df_test

In [ ]:
import re

label_map = {
    'Positive': 2,
    'positive': 2,
    'Negative': 0,
    'negative': 0,
    'Neutral': 1,
    'neutral': 1,
    'None':1
}
def extract_sentiment_word(sentence):
    # Define a regular expression pattern to capture the sentiment word
    sentiment_pattern = re.compile(r'label: (positive|negative|neutral)', re.IGNORECASE)

    # Search for the sentiment pattern in the sentence
    match = sentiment_pattern.search(sentence)

    # If a match is found, extract the sentiment word
    if match:
        sentiment_word = match.group(1).capitalize()  # Capitalize the first letter
        return sentiment_word
    else:
        return None  # Return None if no sentiment word is found

#sentence: Metro Inc. Just Missed Earnings And Its EPS Looked Sad - But Analysts Have Updated Their Models , label: Negative,
device = "cuda:0"
predictions = []  # List to store the numerical labels
for i, row in df_test.iterrows():
    #input_text = """Classify the sentiment of this sentence into one of these categories: Positive, Negative ,or Neutral: """ + row['sentence']
    prompt_template = """Analyze the following sentence and determine if the sentiment is: positive or negative or neutral. Return only a single word, either Positive or Negative or Neutral:
    ###sentence: {query}

    ###label:"""
    prompt = prompt_template.format(query=row['text'])
    #print(prompt)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.unk_token_id )
    decoded = tokenizer.batch_decode(generated_ids)

    #print(input_text)
    print(i)
    #encodeds = tokenizer(input_text, return_tensors="pt", add_special_tokens=True)
    #model_inputs = encodeds.to(device)

    #generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    #decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    print(decoded[0])

    sentiment_word = extract_sentiment_word(decoded[0])
    print(sentiment_word)
    #match1 = re.search(r'###label: (\w+)</s>', )

    # Extract the matched text from the string representation
    #match_text = re.search(r'match=\'(.*?)\'', match1).group(1)

# Use regex to extract the label
    #match = re.search(r'###label: (\w+)', match_text)
    #print(match)
    #if match:
        #sentiment_label = match.group(1)
        #print(sentiment_label)
    #print(sentiment_word)
    #if(sentiment_word =='None'):
      #sentiment_word='Neutral'
    #print(sentiment_word)
    #print(generated_label)
    #numerical_label = label_map[sentiment_label]
    #print(numerical_label)
    predictions.append(sentiment_word)  # Append the numerical label to the list
df_test['predict'] = predictions  # Assign the list of numerical labels to the 'predict' column


In [ ]:
df['predict_0'].value_counts()

In [ ]:
df['predict_0'] = df['predict_0'].fillna('Neutral')

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict'], average='weighted')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict'], average='weighted')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict'], average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict'], average='macro')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict'], average='macro')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict'], average='macro')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

#**1-shot**

In [ ]:
import re

label_map = {
    'Positive': 2,
    'positive': 2,
    'Negative': 0,
    'negative': 0,
    'Neutral': 1,
    'neutral': 1,
    'None':1
}
def extract_sentiment_word(sentence):
    # Define a regular expression pattern to capture the sentiment word
    sentiment_pattern = re.compile(r'label: (positive|negative|neutral)', re.IGNORECASE)

    # Search for the sentiment pattern in the sentence
    match = sentiment_pattern.search(sentence)

    # If a match is found, extract the sentiment word
    if match:
        sentiment_word = match.group(1).capitalize()  # Capitalize the first letter
        return sentiment_word
    else:
        return None  # Return None if no sentiment word is found

#sentence: Metro Inc. Just Missed Earnings And Its EPS Looked Sad - But Analysts Have Updated Their Models , label: Negative,
device = "cuda:0"
predictions1 = []  # List to store the numerical labels
for i, row in df_test.iterrows():
    #input_text = """Classify the sentiment of this sentence into one of these categories: Positive, Negative ,or Neutral: """ + row['sentence']
    prompt_template = """Analyze the following sentence and determine if the sentiment is: positive or negative or neutral\n
    See below all the possible labels and their description\n
    description: Bearish signal\n
    label: Negative\n
    description: neutral sentiment\n
    label: Neutral\n
    description: Bullish sentiment\n
    label: Positive\n
    ###
    See below a couple of examples\n
    sentence: Kohl's stock price target cut to $50 from $53 at Wedbush", label: Negative,\n
    sentence: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing, label: Neutral,\n
    sentence: Adjusted for changes in the Group structure , the Division 's net sales increased by 1.7 %, label: Positive,\n
    Below is the sentence that needs classification. Please provide a single-word response: Positive, Negative, or Neutral\n
    ###sentence: {query}

    ###label:"""
    prompt = prompt_template.format(query=row['text'])
    #print(prompt)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.unk_token_id )
    decoded = tokenizer.batch_decode(generated_ids)

    #print(input_text)
    print(i)
    #encodeds = tokenizer(input_text, return_tensors="pt", add_special_tokens=True)
    #model_inputs = encodeds.to(device)

    #generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    #decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    print(decoded[0])

    # Use regex to extract the label before </s>
    match = re.search(r'###label: (\w+)</s>', decoded[0])

# Check if the match is found and extract the label
    if match:
        extracted_label = match.group(1)
    print(extracted_label)
    #sentiment_word = extract_sentiment_word(decoded[0])
    #print(sentiment_word)
    #match1 = re.search(r'###label: (\w+)</s>', )

    # Extract the matched text from the string representation
    #match_text = re.search(r'match=\'(.*?)\'', match1).group(1)

# Use regex to extract the label
    #match = re.search(r'###label: (\w+)', match_text)
    #print(match)
    #if match:
        #sentiment_label = match.group(1)
        #print(sentiment_label)
    #print(sentiment_word)
    #if(sentiment_word =='None'):
      #sentiment_word='Neutral'
    #print(sentiment_word)
    #print(generated_label)
    #numerical_label = label_map[sentiment_label]
    #print(numerical_label)
    predictions1.append(extracted_label)  # Append the numerical label to the list
df_test['predict_1'] = predictions1  # Assign the list of numerical labels to the 'predict' column


In [ ]:
df_test['predict_1'] = predictions1

In [ ]:
df_test['predict_1'].value_counts()

In [ ]:
data = {'column': ['Neutral', 'Positive', 'Negative', 'neutral', 'Bullish', 'positive', 'Ambivalent', 'negative', 'Label', 'posi', 'N', 'Miss', 'POSITIVE', 'bullish']}

In [ ]:
# Define the mapping
label_mapping = {
    'Neutral': 'Neutral',
    'Positive': 'Positive',
    'Negative': 'Negative',
    'neutral': 'Neutral',
    'Bullish': 'Positive',
    'positive': 'Positive',
    'Ambivalent': 'Neutral',
    'negative': 'Neutral',
    'Label': 'Neutral',
    'posi': 'Neutral',
    'N': 'Neutral',
    'Miss': 'Neutral',
    'POSITIVE': 'Positive',
    'bullish': 'Positive'
}

In [ ]:
df_test['predict_1'] = df_test['predict_1'].replace(label_mapping)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_1'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_1'], average='weighted')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_1'], average='weighted')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_1'], average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_1'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_1'], average='macro')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_1'], average='macro')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_1'], average='macro')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

#**5-shot**

In [ ]:
import re

label_map = {
    'Positive': 2,
    'positive': 2,
    'Negative': 0,
    'negative': 0,
    'Neutral': 1,
    'neutral': 1,
    'None':1
}
def extract_sentiment_word(sentence):
    # Define a regular expression pattern to capture the sentiment word
    sentiment_pattern = re.compile(r'label: (positive|negative|neutral)', re.IGNORECASE)

    # Search for the sentiment pattern in the sentence
    match = sentiment_pattern.search(sentence)

    # If a match is found, extract the sentiment word
    if match:
        sentiment_word = match.group(1).capitalize()  # Capitalize the first letter
        return sentiment_word
    else:
        return None  # Return None if no sentiment word is found

#sentence: Metro Inc. Just Missed Earnings And Its EPS Looked Sad - But Analysts Have Updated Their Models , label: Negative,
device = "cuda:0"
predictions1 = []  # List to store the numerical labels
for i, row in df_test.iterrows():
    #input_text = """Classify the sentiment of this sentence into one of these categories: Positive, Negative ,or Neutral: """ + row['sentence']
    prompt_template = """Analyze the following sentence and determine if the sentiment is: positive or negative or neutral\n
    See below all the possible labels and their description\n
    description: Bearish signal\n
    label: Negative\n
    description: neutral sentiment\n
    label: Neutral\n
    description: Bullish sentiment\n
    label: Positive\n
    ###
    See below a couple of examples\n
    sentence: expected pretax loss of EUR46m in the fourth quarter of 2009 from a year-earlier profit of EUR45m, label: Negative,\n
    sentence: Cemex cut at Credit Suisse, J.P. Morgan on weak building outlook , label: Negative,\n
    sentence: Disney downgraded as analyst says parks attendance could take 2 years to 'normalize', label: Negative,\n
    sentence: Forescout downgraded to neutral from buy at Monness Crespi Hardt", label: "Negative,\n
    sentence: Hanesbrands stock price target cut to $13 vs. $16 at BofA Merrill Lynch , label: Negative,\n
    sentence: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing, label: Neutral,\n
    sentence: ululemon Falls on Conservative View But Analysts Keep Fait", label: Neutral,\n
    sentence: Pyxus International among consumer gainers; Tupperware Brands and Fossil among losers, label: Neutral,\n
    sentence: 12 Technology Stocks Moving In Monday's Pre-Market Session, label: Neutral,\n
    sentence: Amneal Pharmaceuticals leads healthcare gainers; Bellicum Pharmaceuticals among losers, label: Neutral,\n
    sentence: Adjusted for changes in the Group structure , the Division 's net sales increased by 1.7 %, label: Positive,\n
    sentence: Wells Fargo Downgrades Netflix $NFLX to Underperform but sees as a takeover target. NFLX could get acquired, label: Positive,\n
    sentence: Analysts Mostly Bullish On Newly Public Insurance Firm BRP Group, label: Positive,\n
    sentence: arGen-X stock price target raised to $255 vs. $203 at Instinet, label: Positive,\n
    sentence: Baird bullish on beat-up Canada Goose, label: Positive,\n
    Below is the sentence that needs classification. Please provide a single-word response: Positive, Negative, or Neutral\n

    ###sentence: {query}

    ###label:"""
    prompt = prompt_template.format(query=row['text'])
    #print(prompt)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.unk_token_id )
    decoded = tokenizer.batch_decode(generated_ids)

    #print(input_text)
    print(i)
    #encodeds = tokenizer(input_text, return_tensors="pt", add_special_tokens=True)
    #model_inputs = encodeds.to(device)

    #generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    #decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    print(decoded[0])

    # Use regex to extract the label before </s>
    match = re.search(r'###label: (\w+)</s>', decoded[0])

# Check if the match is found and extract the label
    if match:
        extracted_label = match.group(1)
    print(extracted_label)
    #sentiment_word = extract_sentiment_word(decoded[0])
    #print(sentiment_word)
    #match1 = re.search(r'###label: (\w+)</s>', )

    # Extract the matched text from the string representation
    #match_text = re.search(r'match=\'(.*?)\'', match1).group(1)

# Use regex to extract the label
    #match = re.search(r'###label: (\w+)', match_text)
    #print(match)
    #if match:
        #sentiment_label = match.group(1)
        #print(sentiment_label)
    #print(sentiment_word)
    #if(sentiment_word =='None'):
      #sentiment_word='Neutral'
    #print(sentiment_word)
    #print(generated_label)
    #numerical_label = label_map[sentiment_label]
    #print(numerical_label)
    predictions1.append(extracted_label)  # Append the numerical label to the list
df_test['predict_5'] = predictions1  # Assign the list of numerical labels to the 'predict' column


In [ ]:
df_test['predict_5'].value_counts()

In [ ]:
# Define the mapping
label_mapping = {
    'Neutral': 'Neutral',
    'Positive': 'Positive',
    'Negative': 'Negative',
    'neutral': 'Neutral',
    'Bullish': 'Positive',
    'positive': 'Positive',
    'negative': 'Negative',
    'POSITIVE': 'Positive',
    'Unknown': 'Neutral',
    'Indifferent': 'Neutral',
    'NEUTRAL': 'Neutral',

}

In [ ]:
df_test['predict_5'] = df_test['predict_5'].replace(label_mapping)

In [ ]:
df_test['predict_5'].value_counts()

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_5'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_5'], average='weighted')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_5'], average='weighted')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_5'], average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_5'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_5'], average='macro')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_5'], average='macro')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_5'], average='macro')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
df_test

#**10-shot**

In [ ]:
import re

label_map = {
    'Positive': 2,
    'positive': 2,
    'Negative': 0,
    'negative': 0,
    'Neutral': 1,
    'neutral': 1,
    'None':1
}
def extract_sentiment_word(sentence):
    # Define a regular expression pattern to capture the sentiment word
    sentiment_pattern = re.compile(r'label: (positive|negative|neutral)', re.IGNORECASE)

    # Search for the sentiment pattern in the sentence
    match = sentiment_pattern.search(sentence)

    # If a match is found, extract the sentiment word
    if match:
        sentiment_word = match.group(1).capitalize()  # Capitalize the first letter
        return sentiment_word
    else:
        return None  # Return None if no sentiment word is found

#sentence: Metro Inc. Just Missed Earnings And Its EPS Looked Sad - But Analysts Have Updated Their Models , label: Negative,
device = "cuda:0"
predictions1 = []  # List to store the numerical labels
for i, row in df_test.iterrows():
    #input_text = """Classify the sentiment of this sentence into one of these categories: Positive, Negative ,or Neutral: """ + row['sentence']
    prompt_template = """Analyze the following sentence and determine if the sentiment is: positive or negative or neutral\n
    See below all the possible labels and their description\n
    description: Bearish signal\n
    label: Negative\n
    description: neutral sentiment\n
    label: Neutral\n
    description: Bullish sentiment\n
    label: Positive\n
    ###
    See below a couple of examples\n
    See below a couple of examples
    sentence: expected pretax loss of EUR46m in the fourth quarter of 2009 from a year-earlier profit of EUR45m, label: Negative,\n
    sentence: Cemex cut at Credit Suisse, J.P. Morgan on weak building outlook , label: Negative,\n
    sentence: Disney downgraded as analyst says parks attendance could take 2 years to 'normalize', label: Negative,\n
    sentence: Forescout downgraded to neutral from buy at Monness Crespi Hardt", label: "Negative,\n
    sentence: Hanesbrands stock price target cut to $13 vs. $16 at BofA Merrill Lynch , label: Negative,\n
    sentence: Norwegian Cruise Line downgraded to strong sell from hold at CFRA, label: Negative,\n
    sentence: Orbcomm price target lowered to $9 at Canaccord, label": Negative,\n
    sentence: Oppenheimer cuts estimates on Yum China", label: Negative,\n
    sentence: Kohl's stock price target cut to $50 from $53 at Wedbush", label: Negative,\n
    sentence: "2U downgraded to sector weight from overweight at KeyBanc Capital, label: Negative,\n

    sentence: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing, label: Neutral,\n
    sentence: ululemon Falls on Conservative View But Analysts Keep Fait", label: Neutral,\n
    sentence: Pyxus International among consumer gainers; Tupperware Brands and Fossil among losers, label: Neutral,\n
    sentence: 12 Technology Stocks Moving In Monday's Pre-Market Session, label: Neutral,\n
    sentence: Amneal Pharmaceuticals leads healthcare gainers; Bellicum Pharmaceuticals among losers, label: Neutral,\n
    sentence: eyond Meat and Skechers among consumer gainers; Village Farms International among losers, label: Neutral,\n
    sentence: Cameco on the move after Q4 earnings topper, label: Neutral,\n
    sentence: Luckin Coffee shares halted in premarket, label: Neutral,\n
    sentence: he Week's Gainers and Losers on the Stoxx Europe 50 , label: Neutral,\n
    sentence: Stocks making the biggest moves premarket: TD Ameritrade, Charles Schwab, Tesla & more, label: Neutral,\n

    sentence: Adjusted for changes in the Group structure , the Division 's net sales increased by 1.7 %, label: Positive,\n
    sentence: Wells Fargo Downgrades Netflix $NFLX to Underperform but sees as a takeover target. NFLX could get acquired, label: Positive,\n
    sentence: Analysts Mostly Bullish On Newly Public Insurance Firm BRP Group, label: Positive,\n
    sentence: arGen-X stock price target raised to $255 vs. $203 at Instinet, label: Positive,\n
    sentence: Baird bullish on beat-up Canada Goose, label: Positive,\n
    sentence: Bed Bath & Beyond reiterated as buy at BofA Merrill Lynch, label: Positive,\n
    sentence: Best Buy stock price target raised to $105 from $86 at Oppenheimer, label": Positive,\n
    sentence: Berry Petroleum upgraded to sector weight from underweight at KeyBanc Capital, label: Positive,\n
    sentence: Bank of America stock price target raised to $49 from $48 at Oppenheimer", label: Positive,\n
    sentence: Analysts See 18% Gains Ahead For The Holdings Of The First Trust Dow Jones Internet Index Fund ETF, label: Positive,\n
    Below is the sentence that needs classification. Please provide a single-word response: Positive, Negative, or Neutral\n

    ###sentence: {query}

    ###label:"""
    prompt = prompt_template.format(query=row['text'])
    #print(prompt)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.unk_token_id )
    decoded = tokenizer.batch_decode(generated_ids)

    #print(input_text)
    print(i)
    #encodeds = tokenizer(input_text, return_tensors="pt", add_special_tokens=True)
    #model_inputs = encodeds.to(device)

    #generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    #decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    print(decoded[0])

    # Use regex to extract the label before </s>
    match = re.search(r'###label: (\w+)</s>', decoded[0])

# Check if the match is found and extract the label
    if match:
        extracted_label = match.group(1)
    print(extracted_label)
    #sentiment_word = extract_sentiment_word(decoded[0])
    #print(sentiment_word)
    #match1 = re.search(r'###label: (\w+)</s>', )

    # Extract the matched text from the string representation
    #match_text = re.search(r'match=\'(.*?)\'', match1).group(1)

# Use regex to extract the label
    #match = re.search(r'###label: (\w+)', match_text)
    #print(match)
    #if match:
        #sentiment_label = match.group(1)
        #print(sentiment_label)
    #print(sentiment_word)
    #if(sentiment_word =='None'):
      #sentiment_word='Neutral'
    #print(sentiment_word)
    #print(generated_label)
    #numerical_label = label_map[sentiment_label]
    #print(numerical_label)
    predictions1.append(extracted_label)  # Append the numerical label to the list
df_test['predict_10'] = predictions1  # Assign the list of numerical labels to the 'predict' column


In [ ]:
df_test['predict_10'].value_counts()

In [ ]:
# Define the mapping
label_mapping = {
    'Neutral': 'Neutral',
    'Positive': 'Positive',
    'Negative': 'Negative',
    'neutral': 'Neutral',
    'Bullish': 'Positive',
    'Bearish': 'Negative',
    'positive': 'Positive',
    'negative': 'Negative',
    'POSITIVE': 'Positive',
    'Unknown': 'Neutral',
    'Upgrade': 'Neutral',
    'None': 'Neutral',

}

In [ ]:
df_test['predict_10'] = df_test['predict_10'].replace(label_mapping)

In [ ]:
df_test['predict_10'].value_counts()

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_10'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_10'], average='weighted')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_10'], average='weighted')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_10'], average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy
accuracy = accuracy_score(df_test['label'], df_test['predict_10'])

# Calculate precision
precision = precision_score(df_test['label'], df_test['predict_10'], average='macro')

# Calculate recall
recall = recall_score(df_test['label'], df_test['predict_10'], average='macro')

# Calculate F1 score
f1 = f1_score(df_test['label'], df_test['predict_10'], average='macro')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")